# Monarch Watch Data all taggings GBIF

In [1]:
#  Import and read
import pandas as pd 
import numpy as np
import json
# Sue Mottet 2-11-2022

In [2]:
MWatch_Tagged_df = pd.read_csv("MWatch_occurrences_1996_2000.csv")
MWatch_Tagged_df.head()

,id,modified,language,license,institutionID,collectionID,institutionCode,collectionCode,datasetName,basisOfRecord,...,decimalLatitude,decimalLongitude,geodeticDatum,georeferenceRemarks,taxonID,nameAccordingToID,scientificName,kingdom,vernacularName,taxonRemarks
0,MW0000001,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
1,MW0000002,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
2,MW0000003,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
3,MW0000004,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
4,MW0000005,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus


In [3]:
MWatch_Tagged_df.columns.tolist()

['id',
 'modified',
 'language',
 'license',
 'institutionID',
 'collectionID',
 'institutionCode',
 'collectionCode',
 'datasetName',
 'basisOfRecord',
 'occurrenceID',
 'catalogNumber',
 'recordedBy',
 'occurrenceStatus',
 'associatedReferences',
 'eventDate',
 'year',
 'country',
 'stateProvince',
 'verbatimLocality',
 'decimalLatitude',
 'decimalLongitude',
 'geodeticDatum',
 'georeferenceRemarks',
 'taxonID',
 'nameAccordingToID',
 'scientificName',
 'kingdom',
 'vernacularName',
 'taxonRemarks']

In [4]:
def df_to_geojson(df, properties, lat='decimalLatitude', lon='decimalLongitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [5]:
MWatch_Tagged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363582 entries, 0 to 363581
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    363582 non-null  object 
 1   modified              363582 non-null  object 
 2   language              363582 non-null  object 
 3   license               363582 non-null  object 
 4   institutionID         363582 non-null  object 
 5   collectionID          363582 non-null  object 
 6   institutionCode       363582 non-null  object 
 7   collectionCode        363582 non-null  object 
 8   datasetName           363582 non-null  object 
 9   basisOfRecord         363582 non-null  object 
 10  occurrenceID          363582 non-null  object 
 11  catalogNumber         363582 non-null  object 
 12  recordedBy            363582 non-null  object 
 13  occurrenceStatus      363582 non-null  object 
 14  associatedReferences  363582 non-null  object 
 15  

In [6]:
MWatch_Tagged_df.describe()

,year,decimalLatitude,decimalLongitude,taxonID
count,363582.000000,363582.000000,363582.000000,363582.0
mean,1997.858071,39.694501,-90.112869,117273.0
std,1.763730,3.769477,8.696296,0.0
min,1992.000000,25.721490,-118.997683,117273.0
25%,1997.000000,38.203070,-96.731103,117273.0
50%,1998.000000,39.216660,-93.664964,117273.0
75%,1999.000000,42.213930,-82.491268,117273.0
max,2000.000000,49.895140,-67.958850,117273.0


In [7]:
print(MWatch_Tagged_df.shape)

(363582, 30)


In [8]:
# Determine the number of unique values in each column.
for col in MWatch_Tagged_df:
  print(MWatch_Tagged_df[col].unique())

['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['7/30/2019']
['English']
['https://creativecommons.org/publicdomain/zero/1.0/legalcode']
['https://bison.usgs.gov/']
['https://bison.usgs.gov/ipt/resource?r=monarchwatch']
['BISON']
['Monarch Watch']
['Monarch Watch']
['humanobservation']
['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['Monarch Watch']
['present']
['Monarch Watch Tagging Data, https://monarchwatch.org/tagdata']
['8/9/1992' '8/10/1992' '9/1/1992' ... '12/29/2000' '12/30/2000'
 '12/31/2000']
[1992 1993 1994 1995 1996 1997 1998 1999 2000]
['US' 'CA']
['MD' 'MO' 'KS' 'MI' 'KY' 'CT' 'NJ' 'CO' 'TX' 'OH' 'NY' 'MA' 'GA' 'IN'
 'PA' 'NC' 'MN' 'ON' 'NE' 'OK' 'FL' 'NH' 'VT' 'IA' 'IL' 'QC' 'WI' 'AZ'
 'VA' 'ME' 'WV' 'TN' 'SD' 'LA' 'RI' 'DE' 'AL' 'NM' 'AR' 'SC' 'ND' 'DC'
 'MS' 'MB' 'WY' 'BC' 'UT' 'ID']
['City:PLUM POINT' 'City:INDEPENDENCE' 'City

In [9]:
# Determine the number of unique values in each column.
for col in MWatch_Tagged_df:
  print(MWatch_Tagged_df[col].value_counts())

MW0000001    1
MW0242418    1
MW0242394    1
MW0242393    1
MW0242392    1
            ..
MW0121193    1
MW0121192    1
MW0121191    1
MW0121190    1
MW0363582    1
Name: id, Length: 363582, dtype: int64
7/30/2019    363582
Name: modified, dtype: int64
English    363582
Name: language, dtype: int64
https://creativecommons.org/publicdomain/zero/1.0/legalcode    363582
Name: license, dtype: int64
https://bison.usgs.gov/    363582
Name: institutionID, dtype: int64
https://bison.usgs.gov/ipt/resource?r=monarchwatch    363582
Name: collectionID, dtype: int64
BISON    363582
Name: institutionCode, dtype: int64
Monarch Watch    363582
Name: collectionCode, dtype: int64
Monarch Watch    363582
Name: datasetName, dtype: int64
humanobservation    363582
Name: basisOfRecord, dtype: int64
MW0000001    1
MW0242418    1
MW0242394    1
MW0242393    1
MW0242392    1
            ..
MW0121193    1
MW0121192    1
MW0121191    1
MW0121190    1
MW0363582    1
Name: occurrenceID, Length: 363582, dtype: int6

In [10]:
# Find null values
for column in MWatch_Tagged_df.columns:
   print(f"Column {column} has {MWatch_Tagged_df[column].isnull().sum()} null values")

Column id has 0 null values
Column modified has 0 null values
Column language has 0 null values
Column license has 0 null values
Column institutionID has 0 null values
Column collectionID has 0 null values
Column institutionCode has 0 null values
Column collectionCode has 0 null values
Column datasetName has 0 null values
Column basisOfRecord has 0 null values
Column occurrenceID has 0 null values
Column catalogNumber has 0 null values
Column recordedBy has 0 null values
Column occurrenceStatus has 0 null values
Column associatedReferences has 0 null values
Column eventDate has 0 null values
Column year has 0 null values
Column country has 0 null values
Column stateProvince has 0 null values
Column verbatimLocality has 0 null values
Column decimalLatitude has 0 null values
Column decimalLongitude has 0 null values
Column geodeticDatum has 0 null values
Column georeferenceRemarks has 0 null values
Column taxonID has 0 null values
Column nameAccordingToID has 0 null values
Column scienti

In [11]:
# Find duplicate entries
print(f"Duplicate entries: {MWatch_Tagged_df.duplicated().sum()}")

Duplicate entries: 0


# Format file for geoJSON

In [12]:
MWatch_Tagged_df['species'] = 'Monarch Tagged'

In [13]:
MWatch_Tagged_df.head()

,id,modified,language,license,institutionID,collectionID,institutionCode,collectionCode,datasetName,basisOfRecord,...,decimalLongitude,geodeticDatum,georeferenceRemarks,taxonID,nameAccordingToID,scientificName,kingdom,vernacularName,taxonRemarks,species
0,MW0000001,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
1,MW0000002,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
2,MW0000003,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
3,MW0000004,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
4,MW0000005,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged


In [14]:
MWatch_Tagged_df.drop(['modified', 'language', 'license', 'institutionID', 'collectionID', 'institutionCode', 'collectionCode', 'datasetName', 'basisOfRecord', 'occurrenceID', 'catalogNumber', 'recordedBy', 'occurrenceStatus', 'associatedReferences', 'verbatimLocality', 'geodeticDatum', 'georeferenceRemarks', 'taxonID', 'nameAccordingToID','scientificName','kingdom', 'vernacularName','taxonRemarks'], axis=1, inplace=True)
MWatch_Tagged_df.head()

,id,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species
0,MW0000001,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
1,MW0000002,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
2,MW0000003,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
3,MW0000004,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
4,MW0000005,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged


In [15]:
cols = ['eventDate', 'species','country', 'stateProvince', 'decimalLatitude', 'decimalLongitude', 'number']

In [16]:
MWatch_Tagged_df.shape

(363582, 8)

In [17]:
# MWatch_Tagged_df.replace(np.nan,'',regex=True)

In [18]:
MWatch_Tagged_df.shape

(363582, 8)

In [19]:
MWatch_Tagged_df

,id,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species
0,MW0000001,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
1,MW0000002,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
2,MW0000003,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
3,MW0000004,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
4,MW0000005,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
...,...,...,...,...,...,...,...,...
363577,MW0363578,12/30/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363578,MW0363579,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363579,MW0363580,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363580,MW0363581,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged


In [20]:
# MWatch_Tagged_df['column name'] = df['column name'].replace(['old value'],'new value')

In [21]:
MWatch_Tagged_df =MWatch_Tagged_df.groupby(['eventDate', 'year', 'country', 'stateProvince', 'decimalLatitude', 'decimalLongitude', 'species']).size().reset_index()
MWatch_Tagged_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,0
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7


In [22]:
MWatch_Tagged_df.rename( columns={0:'number'} ,inplace=True)
MWatch_Tagged_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7


In [23]:
MWatch_Tagged_year_df = MWatch_Tagged_df
# MWatch_Tagged_year_df['years'] = pd.DatetimeIndex(MWatch_Tagged_year_df['eventDate']).year

# Filter Tagging Data By Year - 1999

In [24]:
# filtering by year
MWatch_Tagged_1999_df = MWatch_Tagged_df[MWatch_Tagged_df['year']==1999]
MWatch_Tagged_1999_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number
8,1/11/1999,1999,US,FL,28.07807,-82.763713,Monarch Tagged,1
12,1/12/1999,1999,US,FL,28.53834,-81.379237,Monarch Tagged,3
18,1/13/1999,1999,US,FL,28.07807,-82.763713,Monarch Tagged,2
46,1/2/1999,1999,US,FL,28.07807,-82.763713,Monarch Tagged,3
56,1/20/1999,1999,US,FL,28.53834,-81.379237,Monarch Tagged,5


In [25]:
MWatch_Tagged_1999_df['eventDate']

8        1/11/1999
12       1/12/1999
18       1/13/1999
46        1/2/1999
56       1/20/1999
           ...    
39905     9/9/1999
39906     9/9/1999
39907     9/9/1999
39908     9/9/1999
39909     9/9/1999
Name: eventDate, Length: 10310, dtype: object

In [26]:
MWatch_Tagged_1999_df.shape

(10310, 8)

In [27]:
MWatch_Tagged_1999_df = pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_df['month'] = pd.DatetimeIndex(MWatch_Tagged_1999_df['eventDate']).month

In [28]:
MWatch_Tagged_1999_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
8,1/11/1999,1999,US,FL,28.07807,-82.763713,Monarch Tagged,1,1
12,1/12/1999,1999,US,FL,28.53834,-81.379237,Monarch Tagged,3,1
18,1/13/1999,1999,US,FL,28.07807,-82.763713,Monarch Tagged,2,1
46,1/2/1999,1999,US,FL,28.07807,-82.763713,Monarch Tagged,3,1
56,1/20/1999,1999,US,FL,28.53834,-81.379237,Monarch Tagged,5,1


In [29]:
MWatch_Tagged_1999_df.drop(['year'], axis=1, inplace=True)

In [30]:
MWatch_Tagged_1999_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
8,1/11/1999,US,FL,28.07807,-82.763713,Monarch Tagged,1,1
12,1/12/1999,US,FL,28.53834,-81.379237,Monarch Tagged,3,1
18,1/13/1999,US,FL,28.07807,-82.763713,Monarch Tagged,2,1
46,1/2/1999,US,FL,28.07807,-82.763713,Monarch Tagged,3,1
56,1/20/1999,US,FL,28.53834,-81.379237,Monarch Tagged,5,1


## January

In [31]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_1_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==1]
MWatch_Tagged_1999_1_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
8,1/11/1999,US,FL,28.07807,-82.763713,Monarch Tagged,1,1
12,1/12/1999,US,FL,28.53834,-81.379237,Monarch Tagged,3,1
18,1/13/1999,US,FL,28.07807,-82.763713,Monarch Tagged,2,1
46,1/2/1999,US,FL,28.07807,-82.763713,Monarch Tagged,3,1
56,1/20/1999,US,FL,28.53834,-81.379237,Monarch Tagged,5,1


In [32]:
MWatch_Tagged_1999_1_df.shape

(16, 8)

In [33]:
MWatch_Tagged_1999_1_df

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
8,1/11/1999,US,FL,28.07807,-82.763713,Monarch Tagged,1,1
12,1/12/1999,US,FL,28.53834,-81.379237,Monarch Tagged,3,1
18,1/13/1999,US,FL,28.07807,-82.763713,Monarch Tagged,2,1
46,1/2/1999,US,FL,28.07807,-82.763713,Monarch Tagged,3,1
56,1/20/1999,US,FL,28.53834,-81.379237,Monarch Tagged,5,1
68,1/23/1999,US,LA,29.95107,-90.071532,Monarch Tagged,1,1
74,1/25/1999,US,TX,29.76043,-95.369803,Monarch Tagged,1,1
76,1/26/1999,US,TX,29.76043,-95.369803,Monarch Tagged,2,1
79,1/27/1999,US,FL,28.53834,-81.379237,Monarch Tagged,3,1
80,1/27/1999,US,TX,29.76043,-95.369803,Monarch Tagged,1,1


In [34]:
MWatch_Tagged_1999_1_df = pd.DataFrame(MWatch_Tagged_1999_1_df)
MWatch_Tagged_1999_1_df.drop(['month'], axis=1, inplace=True)

In [35]:
MWatch_Tagged_1999_1_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
8,1/11/1999,US,FL,28.07807,-82.763713,Monarch Tagged,1
12,1/12/1999,US,FL,28.53834,-81.379237,Monarch Tagged,3
18,1/13/1999,US,FL,28.07807,-82.763713,Monarch Tagged,2
46,1/2/1999,US,FL,28.07807,-82.763713,Monarch Tagged,3
56,1/20/1999,US,FL,28.53834,-81.379237,Monarch Tagged,5


In [36]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_1_df['decimalLatitude'] = MWatch_Tagged_1999_1_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_1_df['decimalLongitude'] = MWatch_Tagged_1999_1_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [37]:
geojson = df_to_geojson(MWatch_Tagged_1999_1_df, cols)

In [38]:
file = open("MWatch_Tagged_1999_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [39]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_2_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==2]
MWatch_Tagged_1999_2_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11533,2/10/1999,US,FL,28.53834,-81.379237,Monarch Tagged,2,2
11538,2/15/1999,US,TX,29.76043,-95.369803,Monarch Tagged,1,2
11542,2/18/1999,US,TX,29.76043,-95.369803,Monarch Tagged,1,2


In [40]:
MWatch_Tagged_1999_2_df.shape

(3, 8)

In [41]:
MWatch_Tagged_1999_2_df = pd.DataFrame(MWatch_Tagged_1999_2_df)
MWatch_Tagged_1999_2_df.drop(['month'], axis=1, inplace=True)

In [42]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_2_df['decimalLatitude'] = MWatch_Tagged_1999_2_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_2_df['decimalLongitude'] = MWatch_Tagged_1999_2_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [43]:
geojson = df_to_geojson(MWatch_Tagged_1999_2_df, cols)

In [44]:
file = open("MWatch_Tagged_1999_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [109]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_3_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==3]
MWatch_Tagged_1999_3_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month


In [110]:
MWatch_Tagged_1999_3_df.shape

(0, 8)

In [111]:
MWatch_Tagged_1999_3_df = pd.DataFrame(MWatch_Tagged_1999_3_df)
MWatch_Tagged_1999_3_df.drop(['month'], axis=1, inplace=True)

In [112]:
MWatch_Tagged_1999_3_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number


In [49]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_3_df['decimalLatitude'] = MWatch_Tagged_1999_3_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_3_df['decimalLongitude'] = MWatch_Tagged_1999_3_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [50]:
geojson = df_to_geojson(MWatch_Tagged_1999_3_df, cols)

In [51]:
file = open("MWatch_Tagged_1999_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [52]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_4_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==4]
MWatch_Tagged_1999_4_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11718,4/29/1999,US,TX,30.26715,-97.743061,Monarch Tagged,2,4
11725,4/30/1999,US,TX,29.87522,-98.262513,Monarch Tagged,1,4
11734,4/6/1999,US,TX,29.50745,-95.094930,Monarch Tagged,2,4
11736,4/7/1999,US,TX,29.50745,-95.094930,Monarch Tagged,1,4
11738,4/8/1999,US,TX,29.50745,-95.094930,Monarch Tagged,1,4


In [53]:
MWatch_Tagged_1999_4_df.shape

(40, 8)

In [54]:
MWatch_Tagged_1999_4_df = pd.DataFrame(MWatch_Tagged_1999_4_df)
MWatch_Tagged_1999_4_df.drop(['month'], axis=1, inplace=True)

In [55]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_4_df['decimalLatitude'] = MWatch_Tagged_1999_4_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_4_df['decimalLongitude'] = MWatch_Tagged_1999_4_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [56]:
geojson = df_to_geojson(MWatch_Tagged_1999_4_df, cols)

In [57]:
file = open("MWatch_Tagged_1999_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [58]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_5_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==5]
MWatch_Tagged_1999_5_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11828,5/5/1999,US,TX,30.26715,-97.743061,Monarch Tagged,1,5
11833,5/6/1999,US,TX,30.26715,-97.743061,Monarch Tagged,3,5
11836,5/7/1999,US,TX,30.26715,-97.743061,Monarch Tagged,1,5
11840,5/8/1999,US,TX,30.26715,-97.743061,Monarch Tagged,1,5
11844,5/9/1999,US,TX,30.26715,-97.743061,Monarch Tagged,2,5


In [59]:
MWatch_Tagged_1999_5_df.shape

(18, 8)

In [60]:
MWatch_Tagged_1999_5_df =pd.DataFrame(MWatch_Tagged_1999_5_df)
MWatch_Tagged_1999_5_df.drop(['month'], axis=1, inplace=True)

In [61]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_5_df['decimalLatitude'] = MWatch_Tagged_1999_5_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_5_df['decimalLongitude'] = MWatch_Tagged_1999_5_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [62]:
geojson = df_to_geojson(MWatch_Tagged_1999_5_df, cols)

In [63]:
file = open("MWatch_Tagged_1999_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [64]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_6_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==6]
MWatch_Tagged_1999_6_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11894,6/5/1999,US,MN,47.73301,-94.548579,Monarch Tagged,1,6
11895,6/5/1999,US,PA,40.67491,-75.617899,Monarch Tagged,73,6
11896,6/6/1999,US,PA,40.67491,-75.617899,Monarch Tagged,8,6
11897,6/7/1999,US,PA,40.74038,-75.309624,Monarch Tagged,7,6
11900,6/9/1999,US,TX,29.50745,-95.094930,Monarch Tagged,1,6


In [65]:
MWatch_Tagged_1999_6_df.shape

(24, 8)

In [66]:
MWatch_Tagged_1999_6_df =pd.DataFrame(MWatch_Tagged_1999_6_df)
MWatch_Tagged_1999_6_df.drop(['month'], axis=1, inplace=True)

In [67]:
MWatch_Tagged_1999_6_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
11851,6/10/1999,US,MN,47.73301,-94.548579,Monarch Tagged,1
11852,6/10/1999,US,TX,29.50745,-95.094930,Monarch Tagged,4
11855,6/12/1999,US,PA,40.67491,-75.617899,Monarch Tagged,19
11858,6/13/1999,US,IL,42.27113,-89.093995,Monarch Tagged,1
11859,6/13/1999,US,TX,29.50745,-95.094930,Monarch Tagged,2


In [68]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_6_df['decimalLatitude'] = MWatch_Tagged_1999_6_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_6_df['decimalLongitude'] = MWatch_Tagged_1999_6_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [69]:
geojson = df_to_geojson(MWatch_Tagged_1999_6_df, cols)

In [70]:
file = open("MWatch_Tagged_1999_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [71]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_7_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==7]
MWatch_Tagged_1999_7_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11933,7/29/1999,US,IN,41.68199,-85.976667,Monarch Tagged,1,7
11934,7/3/1999,US,CT,41.27055,-72.946971,Monarch Tagged,1,7
11935,7/3/1999,US,TX,29.50379,-95.087300,Monarch Tagged,1,7
11943,7/31/1999,US,CT,41.27055,-72.946971,Monarch Tagged,1,7
11944,7/6/1999,US,TX,29.76043,-95.369803,Monarch Tagged,1,7


In [72]:
MWatch_Tagged_1999_7_df.shape

(11, 8)

In [73]:
MWatch_Tagged_1999_7_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11901,7/1/1999,US,CT,41.27055,-72.946971,Monarch Tagged,1,7
11906,7/14/1999,US,MI,43.12003,-85.560032,Monarch Tagged,7,7
11913,7/2/1999,US,TX,29.76043,-95.369803,Monarch Tagged,1,7
11915,7/20/1999,US,ME,43.74353,-70.260176,Monarch Tagged,1,7
11920,7/23/1999,US,MN,47.73301,-94.548579,Monarch Tagged,1,7


In [74]:
MWatch_Tagged_1999_7_df =pd.DataFrame(MWatch_Tagged_1999_7_df)
MWatch_Tagged_1999_7_df.drop(['month'], axis=1, inplace=True)

In [75]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_7_df['decimalLatitude'] = MWatch_Tagged_1999_7_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_7_df['decimalLongitude'] = MWatch_Tagged_1999_7_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [76]:
MWatch_Tagged_1999_7_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
11901,7/1/1999,US,CT,41.27055,-72.946971,Monarch Tagged,1
11906,7/14/1999,US,MI,43.12003,-85.560032,Monarch Tagged,7
11913,7/2/1999,US,TX,29.76043,-95.369803,Monarch Tagged,1
11915,7/20/1999,US,ME,43.74353,-70.260176,Monarch Tagged,1
11920,7/23/1999,US,MN,47.73301,-94.548579,Monarch Tagged,1


In [77]:
geojson = df_to_geojson(MWatch_Tagged_1999_7_df, cols)

In [78]:
file = open("MWatch_Tagged_1999_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [79]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_8_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==8]
MWatch_Tagged_1999_8_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
16146,8/9/1999,US,NY,40.825660,-73.698186,Monarch Tagged,1,8
16147,8/9/1999,US,OH,40.758390,-82.515447,Monarch Tagged,3,8
16148,8/9/1999,US,OH,41.350330,-83.121863,Monarch Tagged,4,8
16149,8/9/1999,US,PA,40.258655,-76.894376,Monarch Tagged,1,8
16150,8/9/1999,US,TX,29.760430,-95.369803,Monarch Tagged,5,8


In [80]:
MWatch_Tagged_1999_8_df.shape

(1615, 8)

In [81]:
MWatch_Tagged_1999_8_df = pd.DataFrame(MWatch_Tagged_1999_8_df)
MWatch_Tagged_1999_8_df.drop(['month'], axis=1, inplace=True)

In [82]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_8_df['decimalLatitude'] = MWatch_Tagged_1999_8_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_8_df['decimalLongitude'] = MWatch_Tagged_1999_8_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [83]:
geojson = df_to_geojson(MWatch_Tagged_1999_8_df, cols)

In [84]:
file = open("MWatch_Tagged_1999_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [85]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_9_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==9]
MWatch_Tagged_1999_9_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
39905,9/9/1999,US,WI,44.740810,-89.063165,Monarch Tagged,3,9
39906,9/9/1999,US,WI,44.875520,-91.919342,Monarch Tagged,6,9
39907,9/9/1999,US,WI,45.306910,-92.362137,Monarch Tagged,1,9
39908,9/9/1999,US,WV,37.801788,-80.445630,Monarch Tagged,2,9
39909,9/9/1999,US,WV,38.642890,-79.331153,Monarch Tagged,26,9


In [86]:
MWatch_Tagged_1999_9_df.shape

(6004, 8)

In [87]:
MWatch_Tagged_1999_9_df = pd.DataFrame(MWatch_Tagged_1999_9_df)
MWatch_Tagged_1999_9_df.drop(['month'], axis=1, inplace=True)

In [88]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_9_df['decimalLatitude'] = MWatch_Tagged_1999_9_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_9_df['decimalLongitude'] = MWatch_Tagged_1999_9_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [89]:
geojson = df_to_geojson(MWatch_Tagged_1999_9_df, cols)

In [90]:
file = open("MWatch_Tagged_1999_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [91]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_10_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==10]
MWatch_Tagged_1999_10_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
10379,10/9/1999,US,VA,38.95863,-77.357003,Monarch Tagged,1,10
10380,10/9/1999,US,WI,42.68279,-89.018722,Monarch Tagged,3,10
10381,10/9/1999,US,WI,42.86000,-90.179850,Monarch Tagged,4,10
10382,10/9/1999,US,WI,45.04357,-92.041015,Monarch Tagged,2,10
10383,10/9/1999,US,WV,39.11511,-79.164759,Monarch Tagged,1,10


In [92]:
MWatch_Tagged_1999_10_df.shape

(2245, 8)

In [93]:
MWatch_Tagged_1999_10_df = pd.DataFrame(MWatch_Tagged_1999_10_df)
MWatch_Tagged_1999_10_df.drop(['month'], axis=1, inplace=True)

In [94]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_10_df['decimalLatitude'] = MWatch_Tagged_1999_10_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_10_df['decimalLongitude'] = MWatch_Tagged_1999_10_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [95]:
geojson = df_to_geojson(MWatch_Tagged_1999_10_df, cols)

In [96]:
file = open("MWatch_Tagged_1999_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [97]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_11_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==11]
MWatch_Tagged_1999_11_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11302,11/9/1999,US,TX,29.50745,-95.094930,Monarch Tagged,2,11
11303,11/9/1999,US,TX,30.34965,-97.370273,Monarch Tagged,1,11
11304,11/9/1999,US,TX,30.50826,-97.678896,Monarch Tagged,1,11
11305,11/9/1999,US,TX,32.75549,-97.330766,Monarch Tagged,1,11
11306,11/9/1999,US,VA,38.83039,-77.196370,Monarch Tagged,1,11


In [98]:
MWatch_Tagged_1999_11_df.shape

(241, 8)

In [99]:
MWatch_Tagged_1999_11_df = pd.DataFrame(MWatch_Tagged_1999_11_df)
MWatch_Tagged_1999_11_df.drop(['month'], axis=1, inplace=True)

In [100]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_11_df['decimalLatitude'] = MWatch_Tagged_1999_11_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_11_df['decimalLongitude'] = MWatch_Tagged_1999_11_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [101]:
geojson = df_to_geojson(MWatch_Tagged_1999_11_df, cols)

In [102]:
file = open("MWatch_Tagged_1999_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [103]:
MWatch_Tagged_1999_month_df =pd.DataFrame(MWatch_Tagged_1999_df)
MWatch_Tagged_1999_12_df = MWatch_Tagged_1999_month_df[MWatch_Tagged_1999_month_df['month']==12]
MWatch_Tagged_1999_12_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11517,12/8/1999,US,TX,29.76043,-95.369803,Monarch Tagged,10,12
11527,12/9/1999,US,FL,28.01863,-82.112864,Monarch Tagged,1,12
11528,12/9/1999,US,FL,28.07807,-82.763713,Monarch Tagged,6,12
11529,12/9/1999,US,TX,29.50745,-95.094930,Monarch Tagged,3,12
11530,12/9/1999,US,TX,29.76043,-95.369803,Monarch Tagged,15,12


In [104]:
MWatch_Tagged_1999_12_df.shape

(93, 8)

In [105]:
MWatch_Tagged_1999_12_df = pd.DataFrame(MWatch_Tagged_1999_12_df)
MWatch_Tagged_1999_12_df.drop(['month'], axis=1, inplace=True)

In [106]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1999_12_df['decimalLatitude'] = MWatch_Tagged_1999_12_df['decimalLatitude'].astype(float)
MWatch_Tagged_1999_12_df['decimalLongitude'] = MWatch_Tagged_1999_12_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [107]:
geojson = df_to_geojson(MWatch_Tagged_1999_12_df, cols)

In [108]:
file = open("MWatch_Tagged_1999_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()